In [ ]:
import os
import cv2
import glob
from PIL import Image, ImageColor
import pandas as pd
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from tqdm.auto import tqdm

def hd_dist(preds, targets):
    preds_coords = np.argwhere(preds) / np.array(preds.shape)
    targets_coords = np.argwhere(targets) / np.array(preds.shape)
    haussdorf_dist = directed_hausdorff(preds_coords, targets_coords)[0]
    return haussdorf_dist

In [ ]:
def gen_random_3dvolume(shape = (100, 100, 100, 3)):
    base = np.zeros(shape, dtype = np.uint8)
    for i in range(3):
        while base.sum() == 0:
            xmin, xmax = sorted(np.random.randint(0, shape[0], 2))
            ymin, ymax = sorted(np.random.randint(0, shape[1], 2))
            zmin, zmax = sorted(np.random.randint(0, shape[2], 2))
            base[xmin:xmax, ymin:ymax, zmin:zmax, i] = 1
    return base

hds = []
dices = []
for i in range(10):
    target = gen_random_3dvolume() # slices * X-size * Y-size * organs
    pred = gen_random_3dvolume() # slices * X-size * Y-size * organs
    
    I = (target & pred).sum((1,2)) # sum by X,Y-size, for image-wise dice
    U = (target | pred).sum((1,2))
    
    hd = 1 - np.mean([hd_dist(pred[...,_], target[...,_]) for _ in range(3)]) # HD of this volume, averaged by organ
    dice = (2 * I / (U + I + 1) + (U == 0)).mean(1) # dice per image, averaged by organ
    hds.append(hd)
    dices.extend(dice)
    
np.mean(dices), np.mean(hds)